# 탐색적 데이터 분석 실시 - 1. 남성경향 제품, 여성경향 제품

## 남녀가 어떤 상품들을 구매하는지, 상품으로써 남녀를 구분해 보고자 한다

 ### process
  #### ch.1
    
    - 1. 상품대분류명 살펴보기(gds_grp_mclas_nm)
    - 2. 상품중분류명 살펴보기(gds_grp_nm)
    - 3. 상품소분류명 살펴보기(goods_id)
    - 4. goods_id에서 남성이 여성보다 더 많이 구매한 물품목록 뽑아내기
    - 5. 뽑아낸 물품목록으로 피쳐 생성하기
    
 ***
  #### ch.2
    - 1. 남성만 혹은 여성만 구매하는 제품을 조사
    - 2. 여성만 구매한 제품목록 뽑아내기
    - 3. 뽑아낸 물품목록으로 피쳐 생성하기
 ***
    -> 뽑아낸 피쳐 저장: train_eda_gender_tendency, test_eda_gender_tendency

### Libraries

In [1]:
# basic 

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100, 'display.max_rows',100)


# plot

import seaborn as sns
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
%matplotlib inline

import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)


## Read Transactions

In [3]:
X_train = pd.read_csv('X_train.csv', encoding='cp949')
y_train = pd.read_csv('y_train.csv', encoding='cp949')
tr = pd.merge(X_train, y_train, on='cust_id')
tr = tr[['cust_id', 'gender', 'tran_date', 'store_nm','goods_id','gds_grp_nm','gds_grp_mclas_nm','amount']]


X_test = pd.read_csv('X_test.csv', encoding='cp949')
tr2 = X_test

trs = [tr, tr2]

*파생변수를 저장할 빈 리스트를 만든다.*

In [4]:
features = [] # 생성된 모든 데이터프레임을 담음
features1 = [] # features에서 train에 해당 되는 피쳐를 담음
features2 = [] # features에서 test에 해당 되는 피쳐를 담음

In [5]:
clothes =['시티웨어', '디자이너', '셔츠', '란제리/내의', '섬유잡화', '캐주얼', '구두', '피혁잡화', '모피/피혁']

clothes2 = ['남성정장', '남성 캐주얼', '남성 트랜디', '커리어', '기타의류']

cosmos = ['화장품', '액세서리']

luxury = ['명품', '준보석/시계', '보석']

sports = ['스포츠','골프', '트래디셔널']

foods = ['축산가공', '건강식품','육류', '수산품', '젓갈/반찬', '가공식품', '농산물', '차/커피' ]

furniture = ['가구', '악기']

etc = ['기타', '침구/수예', '일용잡화', '주방용품', '식기', '주류', '생활잡화']

electronics = ['주방가전', '소형가전', '대형가전', '통신/컴퓨터', '전자/전기']

kids = ['아동', '교복', '문구/팬시']

ambiguous = ['웨딩', '멀티(아울렛)']

all_mclas = [clothes, clothes2, cosmos, luxury, sports, foods, furniture, etc, electronics, kids, ambiguous]

### 4. goods_id에서 남성이 여성보다 더 많이 구매한 물품목록 뽑아내기

   - 남성이 여성보다 구매를 더 많이하면서, 남성이 2회이상 구매한 물품목록을 뽑아내었다
        - 위 조건에 해당하는 goods_id는 313개의 물품이 나왔다

In [6]:
temp_goodsdf = pd.pivot_table(tr, index = 'goods_id', columns='gender', values='amount', aggfunc=np.size, fill_value=0)

temp_goodsdf = temp_goodsdf.rename(columns={0:'women',1:'men'})

temp_goodsdf.query('men > women and men > 2')[:15]

gender,women,men
goods_id,,
5130,26,52
6163,2,3
6188,0,8
8134,4,8
8144,6,12
8190,13,36
8290,0,3
13109,0,20
13114,7,12


In [7]:
# 남성이 더 구매경향을 보이는 goods_id 상품들을 리스트로 뽑아내었다

men_tendency_product = temp_goodsdf.query('men > women and men > 2').index.tolist()

### 5. 뽑아낸 물품목록으로 피쳐 생성하기

    - 0. tr2에서도 이에 해당하는 물품들을 뽑아내고 교집합물품들을 뽑아낸다 -> men_tendency_intersection
    - 1. men_tendency_product에 해당하는 goods_id를 구매하였는지 구매여부
    - 2. men_tendency_product의 구매건수

0. tr2에서도 조건에 해당하는 물품을 뽑아서 교집합하기

In [8]:
men_tendency_product2 = []

for goods_id in men_tendency_product:
    
    if goods_id in tr2.goods_id.unique():
        
        men_tendency_product2.append(goods_id)

In [9]:
# 교집합 품목을 뽑아낸다.
men_tendency_intersection = list(set(men_tendency_product).intersection(men_tendency_product2))

print('교집합 갯수: ', len(men_tendency_intersection))

교집합 갯수:  278


1. 구매여부 -> 구매하였을 경우 10을 주었다

In [15]:
for i in trs:

    temp_goodsdf = pd.pivot_table(i.query('goods_id in @men_tendency_intersection'), index='cust_id', columns='goods_id', values='amount', aggfunc=np.size, fill_value=0)

# 모든 원소에 대해서 만약 원소가 0보다 크다면 10을 주었다 -> (구매안함: 0, 구매함: 10)
    temp_goodsdf = temp_goodsdf = temp_goodsdf.applymap(lambda x : 10 if x>0 else 0)

# 칼럼명 변경(겹치지 않도록)
    origin_name = temp_goodsdf.columns.tolist()
    new_name = [str(i)+'_M_tenden_isbuy' for i in origin_name]
    temp_goodsdf = temp_goodsdf.rename(columns=dict(zip(origin_name, new_name)))
    
    temp_goodsdf = temp_goodsdf.reset_index()

# nan값 제거    
    if len(i) == 232004:
        merge_temp = pd.DataFrame({'cust_id':[i for i in range(3500)]})
    else:
        merge_temp = pd.DataFrame({'cust_id':[i for i in range(3500, 5982)]})
    f = pd.merge(merge_temp, temp_goodsdf, how='outer').fillna(0).astype(int)

    features.append(f); f

In [16]:
f.iloc[:,1:].sum(axis=0)[:20]

5130_M_tenden_isbuy     120
6163_M_tenden_isbuy      10
6188_M_tenden_isbuy      10
8134_M_tenden_isbuy      30
8144_M_tenden_isbuy      20
8290_M_tenden_isbuy      20
13109_M_tenden_isbuy     10
13114_M_tenden_isbuy    100
14198_M_tenden_isbuy     90
26285_M_tenden_isbuy    120
26290_M_tenden_isbuy    100
33288_M_tenden_isbuy     10
34112_M_tenden_isbuy     30
35165_M_tenden_isbuy     10
35265_M_tenden_isbuy     80
37194_M_tenden_isbuy     10
38261_M_tenden_isbuy     30
38263_M_tenden_isbuy     30
39266_M_tenden_isbuy    130
43116_M_tenden_isbuy     60
dtype: int64

2. 구매건수

In [10]:
for i in trs:

    temp_goodsdf = pd.pivot_table(i.query('goods_id in @men_tendency_intersection'), index='cust_id', columns='goods_id', values='amount', aggfunc=np.size, fill_value=0)

# 칼럼명 변경(겹치지 않도록)
    origin_name = temp_goodsdf.columns.tolist()
    new_name = [str(i)+'_M_tendency' for i in origin_name]
    temp_goodsdf = temp_goodsdf.rename(columns=dict(zip(origin_name, new_name)))

    temp_goodsdf = temp_goodsdf.reset_index()
    
# nan값 제거
    if len(i) == 232004:
        merge_temp = pd.DataFrame({'cust_id':[i for i in range(3500)]})
    else:
        merge_temp = pd.DataFrame({'cust_id':[i for i in range(3500, 5982)]})

    f = pd.merge(merge_temp, temp_goodsdf, how='outer').fillna(0).astype(int)

    features.append(f); f

In [18]:
f.iloc[:,1:].sum(axis=0)[:20]

5130_M_tendency     36
6163_M_tendency      1
6188_M_tendency      2
8134_M_tendency      6
8144_M_tendency      4
8290_M_tendency      5
13109_M_tendency     2
13114_M_tendency    12
14198_M_tendency    10
26285_M_tendency    20
26290_M_tendency    32
33288_M_tendency     1
34112_M_tendency    20
35165_M_tendency    32
35265_M_tendency    16
37194_M_tendency     1
38261_M_tendency     3
38263_M_tendency     3
39266_M_tendency    17
43116_M_tendency    18
dtype: int64

## ────────────────────────────────────────────────────────

## ch.2

### 1. 남성만 혹은 여성만 구매하는 제품을 조사

   - 남성들만 혹은 여성들만 구매한 제품군들이 있음을 확인
        - 데이터 불균형을 해소하고자 남성을 구분할 수 있도록 남성들만 쓰는 제품들을 살펴보았으나 goods_id까지 봤지만 데이터가 600여개로 너무 적었음
        - 이에 차라리 데이터가 많은 여성측에서 여성경향이 강한 제품들로 여성을 구분하고자 함

In [11]:
print("---- 전체 ----")

print('대분류: ', tr.gds_grp_mclas_nm.nunique())

print('중분류: ', tr.gds_grp_nm.nunique())

print('소분류: ', tr.goods_id.nunique())

print('\n\n---- 남성 ----')

print('대분류: ', tr.query('gender == 1').gds_grp_mclas_nm.nunique())

print('중분류: ', tr.query('gender == 1').gds_grp_nm.nunique())

print('소분류: ', tr.query('gender == 1').goods_id.nunique())

print('\n\n---- 여성 ----')

print('대분류: ', tr.query('gender == 0').gds_grp_mclas_nm.nunique())

print('중분류: ',tr.query('gender == 0').gds_grp_nm.nunique())

print('소분류: ',tr.query('gender == 0').goods_id.nunique())

---- 전체 ----
대분류:  49
중분류:  324
소분류:  3471


---- 남성 ----
대분류:  46
중분류:  308
소분류:  2618


---- 여성 ----
대분류:  49
중분류:  315
소분류:  3200


## 2. 여성만 쓰는 제품목록 뽑아내기

 - 여성들만 구매하는 제품목록을 대분류 -> 중분류 -> 소분류 순으로 살펴본 결과 대분류, 중분류만으로는 데이터가 너무 적어 구분하기 어렵다고 판단
 - 여성들만 구매하는 소분류(goods_id)제품들을 리스트로 뽑아내었음
     - 조건1: 여성들만 구매
     - 조건2: 여성들이 적어도 3회 이상 구매
     - 조건3: 여성들이 적어도 3명 이상 구매
 - 조건을 모두 만족시키는 제품군을 뽑아서 여성을 구분하는 피쳐를 만들고자 함

조건1

In [12]:
w_use_goods = tr.query('gender == 0').goods_id.unique().tolist()

m_use_goods = tr.query('gender == 1').goods_id.unique().tolist()

all_use_goods = tr.goods_id.unique().tolist()

In [13]:
w_goods = []
for goods in all_use_goods:
     if goods not in m_use_goods:
            w_goods.append(goods)

print('\n여성의 수: ',tr.query('gender == 0').cust_id.nunique())
print('\n여성경향 물품을 구매한 여성 수: ', tr.query('goods_id in @w_goods')['cust_id'].nunique())
print(f'\n여성경향 물품을 구매한 여성 비율: {788 / 2184:.4}')

print('\n여성들만 쓰는 제품수: ', len(w_goods))


여성의 수:  2184

여성경향 물품을 구매한 여성 수:  788

여성경향 물품을 구매한 여성 비율: 0.3608

여성들만 쓰는 제품수:  853


조건2 와 조건3

In [14]:
w_buymorethan3df = tr.query('gender == 0').groupby('cust_id')['goods_id'].value_counts().rename().reset_index().rename(columns={0:'count'})

w_buymorethan3lst = w_buymorethan3df.query('count >= 3').goods_id.unique().tolist()

print('여성들이 3회이상 구매한 물품 목록 수: ', len(w_buymorethan3lst))

w_goodsdf = tr.query('gender == 0').goods_id.value_counts().reset_index().query('goods_id>=3').rename(columns={'index':'goods_id','goods_id':'count'})

w_morethan3pplbuy = w_goodsdf.goods_id.unique().tolist()

print('\n여성들이 3명이상 구매한 물품 목록 수: ', len(w_morethan3pplbuy))

여성들이 3회이상 구매한 물품 목록 수:  1511

여성들이 3명이상 구매한 물품 목록 수:  2367


조건 1,2,3을 모두 만족하는 물품목록(여성만 구매, 여성이 3회이상 구매, 여성이 3명이상 구매)

In [15]:
# 조건1과 조건2 교집합
con1AndCon2 = set(w_goods).intersection(w_buymorethan3lst)

# 교집합된 목록과 조건3의 교집합
w_tendency_product = list(con1AndCon2.intersection(w_morethan3pplbuy))

print('교집합된 물품 목록 수: ', len(w_tendency_product))

교집합된 물품 목록 수:  155


##  3. 뽑아낸 물품목록으로 피쳐 생성하기

 - test데이터도 같은 과정을 거쳐 피쳐를 생성해야하나 gender가 없기 때문에 test데이터의 goods_id와 교집합
 - 둘다 만족하는 최종 w_tendency_product를 토대로 피쳐생성
     - 1. 여성경향 물품 구매건수
     - 2. 여성경향 물품 구매여부

In [16]:
# test데이터의 goods_id 목록
tr2_goodslst = tr2.goods_id.unique().tolist()

# test데이터에도 존재하는 여성경향 몰품 목록 찾기
final_w_tendency_product = []
for product in w_tendency_product:
    
    if product in tr2_goodslst:
        
        final_w_tendency_product.append(product)

# 최종 여성경향 물품목록 추출
print(final_w_tendency_product)
print()
print('개수: ', len(final_w_tendency_product))

[262148, 72204, 221206, 47127, 731171, 562212, 5160, 262189, 303153, 303158, 88120, 88123, 152126, 26179, 67148, 300110, 322128, 641106, 300114, 66136, 87132, 300127, 150114, 257124, 67173, 364135, 43126, 303225, 64124, 364158, 85119, 342143, 322176, 404102, 66186, 85131, 660108, 660106, 85134, 175257, 300195, 85155, 342181, 85160, 300213, 85181, 173245, 303295, 233152, 85187, 807118, 275156, 321247, 84192, 637155, 235243, 423154, 550130, 976118, 43267, 38151, 743177, 191242, 743180, 743181, 740113, 234262, 740120, 274202, 84279, 34106, 16195, 204112, 35152, 674137, 54106, 54110, 38245, 33126, 743281, 354167, 823159, 77178, 821115, 33150, 352130, 54146, 821126, 976263, 740231, 32138, 248198, 165260, 73102, 38287, 676240, 164252, 74147, 546213, 671146, 79273, 461227, 33198, 35247, 671152, 243122, 461234, 740278, 243140, 8135, 740295, 222154, 650186, 35287, 564210, 8183, 8186]

개수:  117


이러한 물품을 구매한 여성의 비율

In [17]:
print('전체 여성 수:', tr.query('gender == 0').cust_id.nunique())

print('\n여성 경향 물품을 구매한 여성 수:', tr.query('goods_id in @final_w_tendency_product').cust_id.nunique())

print(f'\n여성 경향 물품을 구매한 여성의 비율: {350/2184:.4}')

전체 여성 수: 2184

여성 경향 물품을 구매한 여성 수: 350

여성 경향 물품을 구매한 여성의 비율: 0.1603


1. 여성경향 물품 구매건수

In [18]:
for i in trs:

    w_tempdf = pd.pivot_table(i.query('goods_id in @final_w_tendency_product'), index='cust_id', columns='goods_id', values='amount', aggfunc=np.size, fill_value=0)

    origin = w_tempdf.columns.tolist()
    new_name = [str(i)+'_w_tenden' for i in range(len(origin))]
    w_tempdf = w_tempdf.rename(columns=dict(zip(origin, new_name)))

    if len(i) == 232004:
        merge_temp = pd.DataFrame({'cust_id':[i for i in range(3500)]})
    else:
        merge_temp = pd.DataFrame({'cust_id':[i for i in range(3500,5982)]})

    w_tempdf = w_tempdf.reset_index()

    f = pd.merge(merge_temp, w_tempdf, how='outer').fillna(0)

    features.append(f)

2. 여성경향 물품 구매여부

In [27]:
for i in trs:

    w_tempdf = pd.pivot_table(i.query('goods_id in @final_w_tendency_product'), index='cust_id', columns='goods_id', values='amount', aggfunc=np.size, fill_value=0)

# 구매여부 조건: 샀을경우 20 아니면 0
    w_tempdf = w_tempdf.applymap(lambda x: 20 if x > 0 else 0)

    origin = w_tempdf.columns.tolist()
    new_name = [str(i)+'_w_ten_isbuy' for i in range(len(origin))]
    w_tempdf = w_tempdf.rename(columns=dict(zip(origin, new_name)))

    if len(i) == 232004:
        merge_temp = pd.DataFrame({'cust_id':[i for i in range(3500)]})
    else:
        merge_temp = pd.DataFrame({'cust_id':[i for i in range(3500,5982)]})

    w_tempdf = w_tempdf.reset_index()

    f = pd.merge(merge_temp, w_tempdf, how='outer').fillna(0)

    features.append(f)

##  -> 뽑아낸 피쳐 저장: train_eda_gender_tendency, test_eda_gender_tendency

In [19]:
# 남성경향 피쳐: 2(구매여부) + 2(구매건수)
# 여성경향 피쳐: 2(구매여부) + 2(구매건수)

print('8개의 데이터프레임 확인: ', len(features))

8개의 데이터프레임 확인:  4


In [20]:
# 데이터 나누기
for i in features:
    if len(i) == 3500:
        features1.append(i)
    else:
        features2.append(i)
print('train에 들어갈 피쳐 갯수: ', len(features1))
print('test에 들어갈 피쳐 갯수: ', len(features2))

train에 들어갈 피쳐 갯수:  2
test에 들어갈 피쳐 갯수:  2


In [21]:
# train

data = pd.DataFrame({'cust_id': tr.cust_id.unique()})
for f in features :
    data = pd.merge(data, f, how='left').fillna(0)
display(data)
data.to_csv('train_eda_gender_tendency.csv', index=False, encoding='cp949')
data.info()

,cust_id,5130_M_tendency,6163_M_tendency,6188_M_tendency,8134_M_tendency,8144_M_tendency,8290_M_tendency,13109_M_tendency,13114_M_tendency,14198_M_tendency,26285_M_tendency,26290_M_tendency,33288_M_tendency,34112_M_tendency,35165_M_tendency,35265_M_tendency,37194_M_tendency,38261_M_tendency,38263_M_tendency,39266_M_tendency,43116_M_tendency,43132_M_tendency,47147_M_tendency,47204_M_tendency,48190_M_tendency,48233_M_tendency,49124_M_tendency,50127_M_tendency,50146_M_tendency,54115_M_tendency,54186_M_tendency,56109_M_tendency,64113_M_tendency,65204_M_tendency,65205_M_tendency,65217_M_tendency,66101_M_tendency,66131_M_tendency,66185_M_tendency,66280_M_tendency,67141_M_tendency,67142_M_tendency,68108_M_tendency,68144_M_tendency,69128_M_tendency,69169_M_tendency,69170_M_tendency,69174_M_tendency,69175_M_tendency,69245_M_tendency,...,67_w_tenden,68_w_tenden,69_w_tenden,70_w_tenden,71_w_tenden,72_w_tenden,73_w_tenden,74_w_tenden,75_w_tenden,76_w_tenden,77_w_tenden,78_w_tenden,79_w_tenden,80_w_tenden,81_w_tenden,82_w_tenden,83_w_tenden,84_w_tenden,85_w_tenden,86_w_tenden,87_w_tenden,88_w_tenden,89_w_tenden,90_w_tenden,91_w_tenden,92_w_tenden,93_w_tenden,94_w_tenden,95_w_tenden,96_w_tenden,97_w_tenden,98_w_tenden,99_w_tenden,100_w_tenden,101_w_tenden,102_w_tenden,103_w_tenden,104_w_tenden,105_w_tenden,106_w_tenden,107_w_tenden,108_w_tenden,109_w_tenden,110_w_tenden,111_w_tenden,112_w_tenden,113_w_tenden,114_w_tenden,115_w_tenden,116_w_tenden
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,3495,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3496,3496,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3500 entries, 0 to 3499
Columns: 396 entries, cust_id to 116_w_tenden
dtypes: float64(117), int32(278), int64(1)
memory usage: 6.9 MB


In [22]:
# test

data = pd.DataFrame({'cust_id': tr2.cust_id.unique()})
for f in features2 :
    data = pd.merge(data, f, how='left').fillna(0)
display(data)
data.to_csv('test_eda_gender_tendency.csv', index=False, encoding='cp949')
data.info()

,cust_id,5130_M_tendency,6163_M_tendency,6188_M_tendency,8134_M_tendency,8144_M_tendency,8290_M_tendency,13109_M_tendency,13114_M_tendency,14198_M_tendency,26285_M_tendency,26290_M_tendency,33288_M_tendency,34112_M_tendency,35165_M_tendency,35265_M_tendency,37194_M_tendency,38261_M_tendency,38263_M_tendency,39266_M_tendency,43116_M_tendency,43132_M_tendency,47147_M_tendency,47204_M_tendency,48190_M_tendency,48233_M_tendency,49124_M_tendency,50127_M_tendency,50146_M_tendency,54115_M_tendency,54186_M_tendency,56109_M_tendency,64113_M_tendency,65204_M_tendency,65205_M_tendency,65217_M_tendency,66101_M_tendency,66131_M_tendency,66185_M_tendency,66280_M_tendency,67141_M_tendency,67142_M_tendency,68108_M_tendency,68144_M_tendency,69128_M_tendency,69169_M_tendency,69170_M_tendency,69174_M_tendency,69175_M_tendency,69245_M_tendency,...,67_w_tenden,68_w_tenden,69_w_tenden,70_w_tenden,71_w_tenden,72_w_tenden,73_w_tenden,74_w_tenden,75_w_tenden,76_w_tenden,77_w_tenden,78_w_tenden,79_w_tenden,80_w_tenden,81_w_tenden,82_w_tenden,83_w_tenden,84_w_tenden,85_w_tenden,86_w_tenden,87_w_tenden,88_w_tenden,89_w_tenden,90_w_tenden,91_w_tenden,92_w_tenden,93_w_tenden,94_w_tenden,95_w_tenden,96_w_tenden,97_w_tenden,98_w_tenden,99_w_tenden,100_w_tenden,101_w_tenden,102_w_tenden,103_w_tenden,104_w_tenden,105_w_tenden,106_w_tenden,107_w_tenden,108_w_tenden,109_w_tenden,110_w_tenden,111_w_tenden,112_w_tenden,113_w_tenden,114_w_tenden,115_w_tenden,116_w_tenden
0,3500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3501,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3502,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3503,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3504,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,5977,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2478,5978,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2482 entries, 0 to 2481
Columns: 396 entries, cust_id to 116_w_tenden
dtypes: float64(117), int32(278), int64(1)
memory usage: 4.9 MB


# End

## 상품군미지정에 대한 조사
 - 중분류가 '상품군미지정'인 상품들
 - 대분류가 '기타'로 해당되어 있다
 - 이 물품은 특정 분야가 없이 모든 여러 제품들과 함께 지급되는 모양이다.
 - 마치 서비스 물품이나, 행사제품, 기타 제품으로써 특정 물품이 팔릴때 같이 팔리는 것 같다
 
 ### 문제제기
####  - 이 '상품군미지정'이란 제품이 남,여를 구별하는데 방해할 것 같다.           


      * 왜? 사람들은 '상품군미지정'을 사기 위해서 물건을 사는게 아니라       
          원래 다른 물건을 사는데 이것이 데이터로써 같이 저장되서 혼란을 야기하는 것같다.
      * 즉, 데이터분석시 '본 물품' 대신에 '같이 딸려오는 물품'이 명확한 '중분류나, 대분류'없이 중복되어 기록되니      
          제품을 많이 사면 살수록 가장 많이 산 제품이 '상품군미지정'제품 = '기타'제품인것은 당연하다
          
 ### 고려할 점
 - 남,여 등 분석을 위해서는 '상품군미지정'을 드랍시키는 것이 맞지 않을까 싶다.
 - 하지만 마케팅이란 것이 '서비스물품'을 같이 팔아 매출을 올리기도 하니 생각을 해봐야 할 것 같다.       
    왜냐하면 실제로 저 같이 따라오는 '상품군미지정' 제품때문에 본 제품을 살수도 있으니 말이다.
    
    
 ### 시도해볼만한 방안
 - train과 test 데이터 모두 대분류명'기타'를 없앤다
 - '기타'에 해당하는 모든 중분류명 '상품군미지정'을 각각의 '원 제품'의 카테고리로 대분류명을 바꿔준다.
 - '상품군미지정'이라는 이름은 '서비스 및 기타'로 바꾼다.
     * 근데 이거는 머신러닝을 돌렸을 때 유의미한 결과가 나와야 함.
     * 유의미한 결과가 안나오면 모두 포기하고 처음으로 되돌아간다.

In [32]:
tr['gds_grp_mclas_nm'].value_counts()
    # 상품군 미지정이 모든 분야에서 걸쳐서 중복되어서 합계가 나오니 가장 많이 세어지는게 당연할 것이다.

기타         40753
농산물        33169
가공식품       31015
축산가공       12229
수산품        11680
화장품         9305
디자이너        8583
시티웨어        7317
일용잡화        6670
육류          6239
차/커피        5829
섬유잡화        5549
젓갈/반찬       5400
명품          4778
캐주얼         4504
골프          3975
스포츠         3603
주류          3102
아동          2824
구두          2355
란제리/내의      2167
건강식품        2139
주방용품        2091
남성 캐주얼      1774
모피/피혁       1692
피혁잡화        1561
트래디셔널       1534
셔츠          1472
침구/수예       1343
생활잡화        1121
주방가전        1034
커리어          970
남성정장         962
식기           795
액세서리         764
대형가전         405
가구           323
남성 트랜디       228
소형가전         199
통신/컴퓨터       190
보석           164
기타의류          74
준보석/시계        48
악기            43
교복            18
문구/팬시          9
웨딩             3
멀티(아울렛)        1
전자/전기          1
Name: gds_grp_mclas_nm, dtype: int64